In [ ]:
import os
os.environ["PYTORCH_DISABLE_DYNAMO"] = "1"
os.environ["TORCH_COMPILE_DISABLE"] = "1"

# Install PyTorch 2.2.2 with CUDA 11.8 (Colab default)
!pip install torch==2.2.2+cu118 --index-url https://download.pytorch.org/whl/cu118

# Install specific compatible versions of dependencies
!pip install -q git+https://github.com/snap-stanford/deepsnap.git
!pip install -q PyDrive
!pip install ogb
!pip install numpy==1.26.4
!pip install safetensors==0.4.3 # Install a specific version of safetensors

Looking in indexes: https://download.pytorch.org/whl/cu118
  Preparing metadata (setup.py) ... done


In [ ]:
import os

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
# Define a directory for saving, or specify full paths
emb_filename = f"/content/drive/MyDrive/CS145/neurips/FINAL-CODE/artifacts/node_embeddings.pt"


Mounted at /content/drive


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Contrastive link prediction on ogbl-ddi with precomputed negative sampling.
Colab-friendly; robust to Jupyter's extra CLI args. OGB evaluator compliant.

Usage (in Colab):
  !python train_ddi_mlp_contrastive_precomp.py \
      --emb /content/emb.pt --epochs 3 --batch_size 2048 --pool_size 1000
"""

# ---- IMPORTANT: disable Dynamo/compile BEFORE importing torch (Colab quirk) ----
import os
os.environ.setdefault("PYTORCH_DISABLE_DYNAMO", "1")
os.environ.setdefault("TORCH_COMPILE_DISABLE", "1")

import argparse
import json
import random
import re
import warnings
from pathlib import Path
from typing import Dict, Tuple, List

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import math
from math import log, exp



# ---------- OGB import (install on the fly if needed) ----------
def ensure_ogb_installed():
    try:
        import ogb  # noqa: F401
    except Exception:
        import subprocess, sys as _sys
        print("[Info] Installing ogb ...")
        subprocess.check_call([_sys.executable, "-m", "pip", "install", "-q", "ogb"])
    from ogb.linkproppred import LinkPropPredDataset, Evaluator
    return LinkPropPredDataset, Evaluator

LinkPropPredDataset, Evaluator = ensure_ogb_installed()


# ---------------- Utils ----------------

def set_seed(seed: int):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message=re.escape("You are using `torch.load` with `weights_only=False`"),
)

def edge_to_np_pairs(edge) -> Tuple[np.ndarray, np.ndarray]:
    """
    Accepts torch.Tensor [E,2] or numpy.ndarray [E,2].
    Returns u, v as np.int64 arrays.
    """
    if torch.is_tensor(edge):
        assert edge.dim() == 2 and edge.size(1) == 2
        e = edge.cpu().numpy().astype(np.int64)
    else:
        e = np.asarray(edge)
        assert e.ndim == 2 and e.shape[1] == 2
        e = e.astype(np.int64)
    return e[:, 0], e[:, 1]

def build_global_adj(split, num_nodes: int) -> List[set]:
    """Undirected adjacency from train+valid+test to forbid false negatives (for training sampler only)."""
    adj = [set() for _ in range(num_nodes)]
    for part in ("train", "valid", "test"):
        if part in split and "edge" in split[part]:
            u, v = edge_to_np_pairs(split[part]["edge"])
            for a, b in zip(u.tolist(), v.tolist()):
                adj[a].add(b); adj[b].add(a)
    return adj


def precompute_negative_pools(
    adj: List[set],
    num_nodes: int,
    u_nodes: np.ndarray,
    pool_size: int,
    rng: np.random.Generator,
) -> Dict[int, np.ndarray]:
    """
    For each anchor u in u_nodes, precompute a reservoir (<= pool_size) of negatives v
    sampled from V \ ({u} ∪ N(u)). Stored sparsely in a dict for speed/memory.
    """
    all_nodes = np.arange(num_nodes, dtype=np.int64)
    neg_pool: Dict[int, np.ndarray] = {}
    for u in tqdm(np.unique(u_nodes.astype(np.int64)), desc="Precomputing negative pools"):
        forbid = adj[int(u)].copy()
        forbid.add(int(u))
        mask = np.ones(num_nodes, dtype=bool)
        if len(forbid) > 0:
            mask[list(forbid)] = False
        candidates = all_nodes[mask]
        if candidates.size == 0:
            negs = np.empty((0,), dtype=np.int64)
        else:
            take = min(pool_size, candidates.size)
            idx = rng.choice(candidates.size, size=take, replace=False)
            negs = candidates[idx]
        neg_pool[int(u)] = negs.astype(np.int64, copy=False)
    return neg_pool


class RoundRobinPicker:
    """Rolling pointer per u; returns exactly k_neg per u (wraps/shuffles if needed)."""
    def __init__(self, neg_pool: Dict[int, np.ndarray]):
        self.neg_pool = neg_pool
        self.ptr = {u: 0 for u in neg_pool.keys()}

    def pick(self, u_batch: np.ndarray, k_neg: int, rng: np.random.Generator) -> np.ndarray:
        B = u_batch.shape[0]
        out = -np.ones((B, k_neg), dtype=np.int64)
        for i, u in enumerate(u_batch.astype(np.int64).tolist()):
            pool = self.neg_pool.get(u, None)
            if pool is None or pool.size == 0:
                continue  # stays -1
            p = self.ptr[u]
            if p + k_neg <= pool.size:
                sel = pool[p:p+k_neg]
                self.ptr[u] = p + k_neg
                if self.ptr[u] == pool.size:
                    rng.shuffle(pool)
                    self.ptr[u] = 0
            else:
                first = pool[p:]
                rest_need = k_neg - first.size
                rng.shuffle(pool)
                sel = np.concatenate([first, pool[:min(rest_need, pool.size)]])
                if sel.size < k_neg:  # extremely small pool → allow repeats
                    reps = int(np.ceil(k_neg / max(1, pool.size)))
                    sel = np.tile(pool, reps)[:k_neg]
                self.ptr[u] = rest_need % max(1, pool.size)
            out[i, :] = sel.astype(np.int64, copy=False)
        return out


# --------------- Model -----------------

# class Projector(nn.Module):
#     def __init__(self, in_dim: int, hidden: int = 512, proj: int = 256, dropout: float = 0.1):
#         super().__init__()
#         self.net = nn.Sequential(
#             nn.Linear(in_dim, hidden),
#             nn.ReLU(inplace=True),
#             nn.Dropout(dropout),
#             nn.Linear(hidden, proj),
#         )

#     def forward(self, x):
#         z = self.net(x)
#         return F.normalize(z, dim=-1)

class Projector(nn.Module):
    def __init__(self, in_dim: int, hidden: int = 1024, proj: int = 256, dropout: float = 0.1):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, hidden)
        #self.act = nn.GELU()
        self.act = nn.ReLU(inplace=True)
        #self.bn1 = nn.BatchNorm1d(hidden)MAKES IT WORSE
        self.drop = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden, proj)
        # self.ln2 = nn.LayerNorm(proj)

    def forward(self, x):
        h = self.fc1(x)
        h = self.act(h)
        #h = self.bn1(h) if h.dim() == 2 else h   # keep safe for non-2D
        h = self.drop(h)
        z = self.fc2(h)
        #z = self.ln2(z)
        return F.normalize(z, dim=-1)



class Projector_Three(nn.Module):
    def __init__(self, in_dim: int, hidden: int = 1024, proj: int = 256, dropout: float = 0.1):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, hidden)
        self.act = nn.ReLU(inplace=True)
        self.drop = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden, hidden)
        self.fc3 = nn.Linear(hidden, proj)     # final projection

    def forward(self, x):
        h = self.fc1(x)
        h = self.act(h)
        h = self.drop(h)
        h = self.fc2(h)
        h = self.act(h)
        h = self.drop(h)
        z = self.fc3(h)
        return F.normalize(z, dim=-1)




# --------------- OGB-compliant Eval ------------------

@torch.no_grad()
def eval_hits(model, emb, split, batch_size: int, device: torch.device):
    """
    OGB-compliant evaluation for ogbl-ddi.

    For both 'valid' and 'test':
      - y_pred_pos: 1-D (num_pos,)
      - y_pred_neg: 1-D (num_neg,) over a GLOBAL list of negative edges

    We honor whatever format 'edge_neg' provides by converting it to a global [Nneg,2] list:
      * [Nneg, 2]                    -> as is
      * [E, M, 2]                    -> reshape(-1,2)
      * [E, M] (tails only)          -> pair with heads from 'edge' and reshape(-1,2)
      * ragged object rows of tails  -> pair with heads per row and concat
    """
    evaluator = Evaluator(name="ogbl-ddi")

    # Project all nodes once (avoids repeated forward passes)
    Z = model(emb).detach().float().cpu().numpy()  # [N, P]

    def score_pairs_np(pairs_np: np.ndarray, chunk: int = 200_000) -> np.ndarray:
        if pairs_np.size == 0:
            return np.empty((0,), dtype=np.float32)
        u = pairs_np[:, 0].astype(np.int64, copy=False)
        v = pairs_np[:, 1].astype(np.int64, copy=False)
        out = np.empty(u.shape[0], dtype=np.float32)
        for i in range(0, u.shape[0], chunk):
            sl = slice(i, min(i + chunk, u.shape[0]))
            Zu = Z[u[sl]]  # [b, P]
            Zv = Z[v[sl]]  # [b, P]
            out[sl] = np.einsum("bp,bp->b", Zu, Zv, optimize=True).astype(np.float32)
        return out

    def build_split(part: str):
        # Positives → 1-D
        pos = split[part]["edge"]
        pos_np = pos.cpu().numpy().astype(np.int64) if torch.is_tensor(pos) else np.asarray(pos, dtype=np.int64)
        assert pos_np.ndim == 2 and pos_np.shape[1] == 2, "positive edges must be [E,2]"
        y_pred_pos = score_pairs_np(pos_np)  # (E,)

        # Negatives → convert to global [Nneg,2], then score → 1-D
        neg = split[part]["edge_neg"]
        neg_raw = neg.cpu().numpy() if torch.is_tensor(neg) else np.asarray(neg)

        if isinstance(neg_raw, np.ndarray) and neg_raw.ndim == 2 and neg_raw.shape[1] == 2:
            # Canonical DDI: global negative edge list
            neg_pairs = neg_raw.astype(np.int64, copy=False)

        elif isinstance(neg_raw, np.ndarray) and neg_raw.ndim == 3 and neg_raw.shape[-1] == 2:
            # Per-positive pairs -> flatten to global list
            neg_pairs = neg_raw.reshape(-1, 2).astype(np.int64, copy=False)

        elif isinstance(neg_raw, np.ndarray) and neg_raw.ndim == 2:
            # [E, M] tails-only: pair with corresponding heads, then flatten
            E, M = neg_raw.shape
            H = pos_np[:E, 0].astype(np.int64, copy=False)
            heads = np.repeat(H, repeats=M)
            tails = neg_raw.reshape(-1).astype(np.int64, copy=False)
            neg_pairs = np.stack([heads, tails], axis=1)

        elif isinstance(neg_raw, np.ndarray) and (neg_raw.dtype == object or neg_raw.ndim == 1):
            # Ragged rows of tails -> pair per row with head, then concat
            rows = list(neg_raw)
            pairs_list = []
            for i, r in enumerate(rows):
                if r is None:
                    continue
                r = np.asarray(r, dtype=np.int64)
                if r.size:
                    heads = np.full(r.size, pos_np[i, 0], dtype=np.int64)
                    pairs_list.append(np.stack([heads, r], axis=1))
            neg_pairs = np.concatenate(pairs_list, axis=0) if pairs_list else np.empty((0, 2), dtype=np.int64)

        else:
            # Unknown format
            neg_pairs = np.empty((0, 2), dtype=np.int64)

        y_pred_neg = score_pairs_np(neg_pairs)  # (Nneg,)
        return y_pred_pos.astype(np.float32), y_pred_neg.astype(np.float32)

    # Build splits and call the official evaluator (1-D negatives)
    ypp_v, ypn_v = build_split("valid")
    ypp_t, ypn_t = build_split("test")

    # Guard rails: enforce pure 1-D arrays
    ypp_v = np.asarray(ypp_v, dtype=np.float32).reshape(-1)
    ypn_v = np.asarray(ypn_v, dtype=np.float32).reshape(-1)
    ypp_t = np.asarray(ypp_t, dtype=np.float32).reshape(-1)
    ypn_t = np.asarray(ypn_t, dtype=np.float32).reshape(-1)

    val_res  = evaluator.eval({"y_pred_pos": ypp_v, "y_pred_neg": ypn_v})
    test_res = evaluator.eval({"y_pred_pos": ypp_t, "y_pred_neg": ypn_t})
    return val_res, test_res


# --------------- Train -----------------

# def build_parser():
#     ap = argparse.ArgumentParser()
#     ap.add_argument("--emb", type=str, default=emb_filename)
#     ap.add_argument("--dataset_root", type=str, default="./ogb_data")
#     ap.add_argument("--out_dir", type=str, default="./ddi_mlp_contrastive_precomp")
#     ap.add_argument("--epochs", type=int, default=100)
#     ap.add_argument("--batch_size", type=int, default=4096)
#     ap.add_argument("--k_neg", type=int, default=100)
#     ap.add_argument("--pool_size", type=int, default=2000, help="Neg reservoir size per u")
#     ap.add_argument("--hidden", type=int, default=512)
#     ap.add_argument("--proj", type=int, default=256)
#     ap.add_argument("--dropout", type=float, default=0.1)
#     ap.add_argument("--tau", type=float, default=0.07)
#     ap.add_argument("--symmetric", type=int, default=0, help="0=one-sided InfoNCE, 1=also v->u")
#     ap.add_argument("--lr", type=float, default=1e-3)
#     ap.add_argument("--weight_decay", type=float, default=1e-4)
#     ap.add_argument("--device", type=str, default="cuda" if torch.cuda.is_available() else "cpu")
#     ap.add_argument("--seed", type=int, default=42)
#     return ap

# tau = 0.7, lr = 5e-3, k_neg = 400, pool_size = 4000 -> best epoch at 5 (hits@20=0.4705) without eta_min in cosine scheduler
# tau = 0.7, lr = 1e-4, k_neg = 400, pool_size = 4000 -> best epoch at 16 (hits@20=0.5148) with eta_min=args.lr * 0.05 in cosine scheduler
def build_parser():
    ap = argparse.ArgumentParser()
    ap.add_argument("--emb", type=str, default=emb_filename)
    ap.add_argument("--dataset_root", type=str, default="./ogb_data")
    ap.add_argument("--out_dir", type=str, default="/content/drive/MyDrive/CS145/neurips/FINAL-CODE/results/main_ablations/semantic_only")

    ap.add_argument("--epochs", type=int, default=500)
    ap.add_argument("--batch_size", type=int, default=4096)
    ap.add_argument("--k_neg", type=int, default=200)
    # ap.add_argument("--pool_size", type=int, default=4500, help="Neg reservoir size per u")
    ap.add_argument("--pool_size", type=int, default=3000, help="Neg reservoir size per u")
    ap.add_argument("--hidden", type=int, default=512)
    ap.add_argument("--proj", type=int, default=256)
    ap.add_argument("--dropout", type=float, default=0.05)
    ap.add_argument("--tau", type=float, default=0.07)
    ap.add_argument("--symmetric", type=int, default=1, help="0=one-sided InfoNCE, 1=also v->u")
    #ap.add_argument("--lr", type=float, default=5e-3)
    ap.add_argument("--lr", type=float, default=1e-4)
    ap.add_argument("--weight_decay", type=float, default=5e-4)
    ap.add_argument("--device", type=str, default="cuda" if torch.cuda.is_available() else "cpu")
    ap.add_argument("--seed", type=int, default=42)
    return ap

def main(args):
    num_layers = 3
    set_seed(args.seed)
    device = torch.device(args.device)
    out_dir = Path(args.out_dir); out_dir.mkdir(parents=True, exist_ok=True)

    # Data
    dataset = LinkPropPredDataset(name="ogbl-ddi", root=args.dataset_root)
    split = dataset.get_edge_split()

    train_edges = split["train"]["edge"]  # [E,2]
    u_train_np, v_train_np = edge_to_np_pairs(train_edges)

    # Embeddings
    emb = torch.load(args.emb, map_location="cpu")
    if not torch.is_tensor(emb):
        emb = torch.tensor(emb)
    emb = emb.float().to(device)
    num_nodes, in_dim = emb.shape

    # Adjacency & negative pools (for training sampler only)
    adj = build_global_adj(split, num_nodes)
    rng = np.random.default_rng(args.seed)
    neg_pool = precompute_negative_pools(adj, num_nodes, u_train_np, pool_size=args.pool_size, rng=rng)

    #============================================
    #print negative pool stats
    sizes = np.array([len(v) for v in neg_pool.values()])

    print(f"#anchors: {len(sizes)}")
    print(f"min={sizes.min()}  max={sizes.max()}  median={np.median(sizes):.1f}  mean={sizes.mean():.1f}")
    print("p05/25/50/75/95 =", np.percentile(sizes, [5,25,50,75,95]).astype(int))

    # how many anchors have empty pools or too few for k_neg?
    k = args.k_neg
    num_empty = (sizes == 0).sum()
    lt_k = (sizes < k).sum()
    print(f"empty pools: {num_empty}  (<k_neg: {lt_k})  (k_neg={k}, pool_size={args.pool_size})")
    #=============================================

    picker = RoundRobinPicker(neg_pool)

    # Model/opt
    if num_layers == 2:
        model = Projector(in_dim=in_dim, hidden=args.hidden, proj=args.proj, dropout=args.dropout).to(device)
    else:
        model = Projector_Three(in_dim=in_dim, hidden=args.hidden, proj=args.proj, dropout=args.dropout).to(device)

    # Replace fixed temperature with learnable
    ##################
    #opt = torch.optim.AdamW(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    ##opt = torch.optim.AdamW(model.parameters(), lr=args.lr)
    # Learnable temperature (stored as log(1/tau))
    logit_scale = torch.nn.Parameter(torch.tensor(math.log(1/args.tau), dtype=torch.float32, device=device))

    # Include it in the optimizer without weight decay
    opt = torch.optim.AdamW(
                  [{"params": model.parameters(), "weight_decay": args.weight_decay},
                  {"params": [logit_scale],      "weight_decay": 0.0}],
                  lr=args.lr,
              )

    ###################

    # Uniquify in case any tensor appears in multiple param groups
    params_in_opt = {p for g in opt.param_groups for p in g['params']}
    total_opt = sum(p.numel() for p in params_in_opt)
    trainable_opt = sum(p.numel() for p in params_in_opt if p.requires_grad)
    print(f"Params in optimizer: {total_opt:,}  ({total_opt/1e6:.2f}M), "
      f"trainable: {trainable_opt:,}  ({trainable_opt/1e6:.2f}M)")





    warmup_epochs = 10
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=max(1, args.epochs - warmup_epochs), eta_min=args.lr * 0.05)


    order = np.arange(u_train_np.shape[0], dtype=np.int64)
    best_val = float("-inf")
    best_path = out_dir / "model_best.pt"

    for epoch in range(1, args.epochs + 1):
        np.random.shuffle(order)
        model.train()
        running = 0.0; steps = 0

        for i in tqdm(range(0, order.size, args.batch_size), desc=f"Epoch {epoch}/{args.epochs}"):
            idx = order[i:i+args.batch_size]
            u_batch = u_train_np[idx]
            v_pos   = v_train_np[idx]

            def pick_hard_negatives(u_batch_np, pool_dict, emb, k, device):
              # emb is the (possibly trainable) node embedding tensor on device
              U = torch.from_numpy(u_batch_np).long().to(device)
              eu = emb[U]                      # [B, D]
              out = np.empty((U.size(0), k), dtype=np.int64)
              for i, u in enumerate(U.tolist()):
                  pool = pool_dict.get(u, None)
                  if pool is None or pool.size == 0:
                      out[i].fill(-1)
                      continue
                  P = torch.from_numpy(pool).long().to(device)
                  ev = emb[P]                  # [|pool|, D]
                  # score by current projector (hardness under model; best)
                  with torch.no_grad():
                      zu = model(eu[i:i+1])            # [1, P]
                      zv = model(ev)                    # [|pool|, P]
                      s = (zu @ zv.T).squeeze(0)       # [|pool|]
                      topk = torch.topk(s, k=min(k, s.numel()), largest=True).indices
                  sel = P[topk].detach().cpu().numpy()
                  # pad if pool smaller than k
                  if sel.size < k:
                      sel = np.pad(sel, (0, k - sel.size), constant_values=-1)
                  out[i] = sel
              return out

            # use it
            #v_neg = pick_hard_negatives(u_batch, neg_pool, emb, args.k_neg, device)

            v_neg = picker.pick(u_batch, args.k_neg, rng)   # [B, k_neg]

            keep = (v_neg[:, 0] != -1)
            if not np.any(keep):
                continue
            u_batch = u_batch[keep]; v_pos = v_pos[keep]; v_neg = v_neg[keep]

            B = u_batch.shape[0]

            u_t    = torch.from_numpy(u_batch.astype(np.int64)).long().to(device)
            vpos_t = torch.from_numpy(v_pos.astype(np.int64)).long().to(device)
            vneg_t = torch.from_numpy(v_neg.reshape(-1).astype(np.int64)).long().to(device)

            eu = emb[u_t]                               # [B, D]
            ev_pos = emb[vpos_t]                        # [B, D]
            ev_neg = emb[vneg_t].reshape(B, args.k_neg, -1)  # [B, k, D]

            zu = model(eu)                               # [B, P]
            zv_pos = model(ev_pos)                       # [B, P]
            zv_neg = model(ev_neg.reshape(-1, ev_neg.shape[-1])).reshape(B, args.k_neg, -1)

            pos = torch.sum(zu * zv_pos, dim=-1, keepdim=True)     # [B,1]
            neg = torch.einsum("bd,bkd->bk", zu, zv_neg)           # [B,k]



            # Replace fixed temperature with learnable
            ################
            #logits = torch.cat([pos, neg], dim=1) / args.tau
            scale = logit_scale.clamp(min=math.log(1/100), max=math.log(100)).exp()
            if i == 0:
                print(f"  temp ≈ {1/scale.item():.4f} (scale={scale.item():.2f})")
            logits = scale * torch.cat([pos, neg], dim=1)
            ################



            labels = torch.zeros(B, dtype=torch.long, device=device)

            loss = F.cross_entropy(logits, labels)

            if args.symmetric:
                if not hasattr(picker, "neg_pool_v"):
                    picker.neg_pool_v = precompute_negative_pools(adj, num_nodes, v_train_np, args.pool_size, rng)
                    picker.picker_v = RoundRobinPicker(picker.neg_pool_v)
                u_neg = picker.picker_v.pick(v_pos, args.k_neg, rng)  # [B,k]
                keep2 = (u_neg[:, 0] != -1)
                if np.any(keep2):
                    u2 = u_batch[keep2]
                    v2 = v_pos[keep2]
                    u_neg2 = u_neg[keep2]

                    v2_t   = torch.from_numpy(v2.astype(np.int64)).long().to(device)
                    upos_t = torch.from_numpy(u2.astype(np.int64)).long().to(device)
                    uneg_t = torch.from_numpy(u_neg2.reshape(-1).astype(np.int64)).long().to(device)

                    ev_anchor = emb[v2_t]
                    eu_pos2   = emb[upos_t]
                    eu_neg2   = emb[uneg_t].reshape(u2.shape[0], args.k_neg, -1)

                    zv = model(ev_anchor)
                    zu_pos2 = model(eu_pos2)
                    zu_neg2 = model(eu_neg2.reshape(-1, eu_neg2.shape[-1])).reshape(u2.shape[0], args.k_neg, -1)

                    pos2 = torch.sum(zv * zu_pos2, dim=-1, keepdim=True)
                    neg2 = torch.einsum("bd,bkd->bk", zv, zu_neg2)

                    # Replace fixed temperature with learnable
                    ################
                    #logits2 = torch.cat([pos2, neg2], dim=1) / args.tau
                    logits2 = scale * torch.cat([pos2, neg2], dim=1)
                    ################

                    labels2 = torch.zeros(u2.shape[0], dtype=torch.long, device=device)
                    loss = 0.5 * (loss + F.cross_entropy(logits2, labels2))

            opt.zero_grad(set_to_none=True)
            loss.backward()
            opt.step()

            running += float(loss.item()); steps += 1

        # --- warmup ---
        if epoch <= warmup_epochs:
            for g in opt.param_groups:
                g['lr'] = args.lr * epoch / warmup_epochs
        else:
            scheduler.step()

        # (optional) log current LR
        # if epoch == 1 or epoch % 5 == 0:
        #     print(f"lr={opt.param_groups[0]['lr']:.3e}")
        avg_loss = running / max(1, steps)

        # Eval
        model.eval()
        val_res, test_res = eval_hits(model, emb, split, batch_size=32768, device=device)
        val_hits  = {k: v for k, v in val_res.items()  if k.startswith("hits@")}
        test_hits = {k: v for k, v in test_res.items() if k.startswith("hits@")}
        primary = "hits@20" if "hits@20" in val_hits else f"hits@{sorted(int(k.split('@')[1]) for k in val_hits)[0]}"
        primary_test = "hits@20" if "hits@20" in test_hits else f"hits@{sorted(int(k.split('@')[1]) for k in test_hits)[0]}"

        def fmt(d):
            ks = sorted(d.keys(), key=lambda x: int(x.split('@')[1]))
            return " ".join(f"{k}={d[k]:.4f}" for k in ks)

        #print(f"[Epoch {epoch}] loss={avg_loss:.4f} | val: {fmt(val_hits)} | test: {fmt(test_hits)}")

        if float(val_hits.get(primary, -1.0)) > best_val:
            best_val = float(val_hits[primary])
            best_test = test_hits[primary_test]
            # torch.save({"model": model.state_dict(),
            #             "config": vars(args),
            #             "val": val_res, "test": test_res,
            #             "primary": primary}, best_path)
            torch.save({"epoch": epoch,
                            "best_val": best_val,
                            "best_test": best_test,
                            "model": model.state_dict(),
                            "opt": opt.state_dict(),
                            "scheduler": scheduler.state_dict(),
                            "logit_scale": float(logit_scale.detach().cpu()),   # keep learned τ
                            "config": vars(args),
                            "val": val_res,
                            "test": test_res,
                            "rng": {
                                    "py": random.getstate(),
                                    "np": np.random.get_state(),
                                    "torch": torch.get_rng_state(),
                                    "cuda": torch.cuda.get_rng_state_all() if torch.cuda.is_available() else None,
                                  },
                            "primary": primary},
                            best_path)
            print(f"Epoch {epoch}: Best Val {best_val}, Best Test {best_test}")



    with torch.no_grad():
        raw_logit_scale = float(logit_scale.detach().cpu().item())
        raw_scale = float(math.exp(raw_logit_scale))
        raw_tau   = float(math.exp(-raw_logit_scale))

    final_lr = float(opt.param_groups[0]["lr"])
    final_metrics = {
        "config": vars(args),
        "primary": primary,            # which metric you tracked (e.g., hits@20)
        "best_val": best_val,          # best validation score seen
        "best_test": best_test,        # best test score seen
        "final": {                     # end-of-run snapshot
            "epoch": args.epochs,
            "tau_raw": raw_tau,
            "scale_raw": raw_scale,
            "lr": final_lr,
        },
    }
    with open(out_dir / "metrics.json", "w") as f:
        json.dump(final_metrics, f, indent=2)
    print(json.dumps(final_metrics, indent=2))
    print(f"Best Val {best_val}, Best Test {best_test} Done.")



if __name__ == "__main__":
    parser = build_parser()
    # Use parse_known_args so Colab's hidden "-f ..." doesn't crash argparse
    args, _ = parser.parse_known_args()
    main(args)


<>:99: SyntaxWarning: invalid escape sequence '\ '
<>:99: SyntaxWarning: invalid escape sequence '\ '
/tmp/ipython-input-45958657.py:99: SyntaxWarning: invalid escape sequence '\ '
  sampled from V \ ({u} ∪ N(u)). Stored sparsely in a dict for speed/memory.


Downloaded 0.04 GB: 100%|██████████| 46/46 [00:01<00:00, 24.70it/s]


Extracting ./ogb_data/ddi.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 54.81it/s]

Saving...


...

  temp ≈ 0.0100 (scale=100.00)


Epoch 419/500:   0%|          | 0/261 [00:00<?, ?it/s]

  temp ≈ 0.0100 (scale=100.00)


Epoch 419/500:  43%|████▎     | 111/261 [00:54<01:13,  2.04it/s]


KeyboardInterrupt: 